In [ ]:
%env YASSGENES=../161027_yass_allgenes/yassbed12_genomic.bed.gz

In [ ]:
%env YASSOPEAKS=../161027_yass_uvclap_outsidegenes/yassbed12_genomic.bed.gz

In [ ]:
%env PEAKS=../161027_yass_uvclap_outsidegenes/peaks.bed

In [ ]:
%env GENES=../161027_yass_uvclap_outsidegenes/genes.bed

In [ ]:
%env OPEAKSREGIONS=../161027_yass_uvclap_outsidegenes/peaks_outside_genes_slopped_merged.bed

In [ ]:
%%bash

# get precalculated YASS results

cp -v $YASSGENES yass_ongenes.bed.gz
cp -v $YASSOPEAKS yass_onotherpeaks.bed.gz
cp -v $PEAKS peaks.bed
cp -v $GENES genes.bed
cp -v $OPEAKSREGIONS .

In [ ]:
%%bash

wc -l peaks.bed genes.bed

In [ ]:
%%bash

# assign unique ids to YASS pairs and split into q and s parts

for BEDGZ in yass_onotherpeaks.bed.gz yass_ongenes.bed.gz 
do
    zcat $BEDGZ | awk 'BEGIN{OFS="\t"}/^#/{print}!/^#/{$4="yasspair_"i++" "$4; print}' | gzip > $BEDGZ.uid
    # extract first alignment
    zcat $BEDGZ.uid |
    awk 'BEGIN{OFS="\t"}!/^#/{
        chr=$1; start=$14; end=$15; id=$4; score=$18; strand=$7; print chr,start,end,id,score,strand
    }' | sort -k1,1 -k2,2n | gzip > $BEDGZ.partsA.gz
    # extract second alignment
    zcat $BEDGZ.uid |
    awk 'BEGIN{OFS="\t"}!/^#/{
        chr=$1; start=$16; end=$17; id=$4; score=$18; strand=$7; print chr,start,end,id,score,strand
    }' | sort -k1,1 -k2,2n | gzip > $BEDGZ.partsB.gz
done

In [ ]:
%%bash

ls *.parts?.gz

In [ ]:
%%bash

# select genes with peak and at least 3 Alu elements

bedtools intersect -s -u -a genes.bed -b peaks.bed |
bedtools sort > genes_with_peak.bed
wc -l genes_with_peak.bed

bedtools intersect -c \
-a genes_with_peak.bed \
-b /home/maticzkd/genomes/hg19/RepeatMasker_hg19_20161020_Alu.bed | \
awk '$7>=3' | \eval_uvclap_updatealudists_v2
cut -f 1-6 > genes_with_peak_minalus.bed
wc -l genes_with_peak_minalus.bed

In [ ]:
%%bash

# select outside regions with at least 3 Alu elements

bedtools intersect -c \
-a peaks_outside_genes_slopped_merged.bed \
-b /home/maticzkd/genomes/hg19/RepeatMasker_hg19_20161020_Alu.bed | \
awk '$7>=3' | \
cut -f 1-6 > peaks_outside_genes_slopped_merged_minalus.bed

wc -l peaks_outside_genes_slopped_merged_minalus.bed
wc -l peaks_outside_genes_slopped_merged.bed

In [ ]:
%%bash

# assign unique ids to Alus
cat /home/maticzkd/genomes/hg19/RepeatMasker_hg19_20161020_Alu.bed | \
awk 'BEGIN{OFS="\t"}{$4=$4"_id_"i++; print}' |
gzip > alus_uid.bed.gz

In [ ]:
%%bash

# Alus on selected genes
bedtools intersect -u \
-a alus_uid.bed.gz \
-b genes_with_peak_minalus.bed |
gzip > alus_uid_ongenes.bed.gz

In [ ]:
%%bash

# retrieve all alus that are overlapped at least 75% of their length by a yass alignment
# and the corresponding yass pairs

bedtools intersect -loj -f 0.75 \
-a alus_uid_ongenes.bed.gz \
-b yass_ongenes.bed.gz.partsA.gz | gzip > alus_and_yass_pairs_ongenesA.bed.gz

In [ ]:
%%bash

# retrieve all alus that are overlapped at least 75% of their length by a yass alignment
# and the corresponding yass pairs

bedtools intersect -loj -f 0.75 \
-a alus_uid_ongenes.bed.gz \
-b yass_ongenes.bed.gz.partsB.gz | gzip > alus_and_yass_pairs_ongenesB.bed.gz

In [ ]:
%%bash

# Alus on outside regions
bedtools intersect -u \
-a alus_uid.bed.gz \
-b peaks_outside_genes_slopped_merged_minalus.bed |
gzip > alus_uid_offgenes.bed.gz

In [ ]:
%%bash

# retrieve all alus that are overlapped at least 75% of their length by a yass alignment
# and the corresponding yass pairs

bedtools intersect -loj -f 0.75 \
-a alus_uid_offgenes.bed.gz \
-b yass_ongenes.bed.gz.partsA.gz | gzip > alus_and_yass_pairs_outsideA.bed.gz

In [ ]:
%%bash

# retrieve all alus that are overlapped at least 75% of their length by a yass alignment
# and the corresponding yass pairs

bedtools intersect -loj -f 0.75 \
-a alus_uid_offgenes.bed.gz \
-b yass_ongenes.bed.gz.partsB.gz | gzip > alus_and_yass_pairs_outsideB.bed.gz

In [ ]:
%%bash

# no memory footprint calculation of minimal distance yass pairs

for F in alus_and_yass_pairs_ongenes alus_and_yass_pairs_outside
do

for EXT in A.bed.gz B.bed.gz
do
    zcat $F$EXT | 
    sort -k10,10 | 
    awk '$10!="."' | 
    cut -f 1,2,3,4,6,10 > $F$EXT.cleaned
done

join ${F}A.bed.gz.cleaned ${F}B.bed.gz.cleaned -j 6 -t $'\t' | 
awk 'BEGIN{OFS="\t"} \
$5!=$10{ \
    da=$3-$9; \
    db=$8-$4; \
    distance = (da > db ? da : db); \
    print $2,$3,$4,$5,distance,$6,$1; \
    print $7,$8,$9,$10,distance,$11,$1}' | 
    sort -k4,4 -k5,5n |
awk 'BEGIN{OFS="\t"}h[$4]==0{print $1,$2,$3,$4"$"$7,$5,$6; h[$4]=1}' > $F.bed.gz.mindist.bed
#awk '{if (h[$4!=1]) {print $0}}'

done
    
#print "ids",$6,$11, $4,$5, $9,$10, da, db, distance} \

In [ ]:
%%bash

# for each repeat, determine if hit by peak slopped by 100 nt

cat peaks.bed | 
bedtools slop -g ~/genomes/hg19.genome -i - -b 100 |
bedtools intersect -u \
-a alus_and_yass_pairs_ongenes.bed.gz.mindist.bed -b - > genes_hit.bed

cat peaks.bed | 
bedtools slop -g ~/genomes/hg19.genome -i - -b 100 |
bedtools intersect -v \
-a alus_and_yass_pairs_ongenes.bed.gz.mindist.bed -b - > genes_nohit.bed

cat peaks.bed | 
bedtools slop -g ~/genomes/hg19.genome -i - -b 100 |
bedtools intersect -u \
-a alus_and_yass_pairs_outside.bed.gz.mindist.bed -b - > outside_hit.bed

cat peaks.bed | 
bedtools slop -g ~/genomes/hg19.genome -i - -b 100 |
bedtools intersect -v \
-a alus_and_yass_pairs_outside.bed.gz.mindist.bed -b - > outside_nohit.bed

wc -l genes_hit.bed
wc -l genes_nohit.bed
wc -l outside_hit.bed
wc -l outside_nohit.bed

In [ ]:
import rpy2
%load_ext rpy2.ipython

In [ ]:
%%R

library(dplyr)

In [ ]:
%%R

# load targeted and non-targeted Alus with distances

d <- bind_rows(
    read.table(
        "genes_hit.bed", 
        sep="\t",
        col.names=c("chr","start","stop","id","score","strand")) %>%
        mutate(source="genes", hit=T),
    read.table(
        "genes_nohit.bed", 
        sep="\t",
        col.names=c("chr","start","stop","id","score","strand")) %>%
        mutate(source="genes",hit=F),
    read.table(
        "outside_hit.bed", 
        sep="\t",
        col.names=c("chr","start","stop","id","score","strand")) %>%
        mutate(source="outside", hit=T),
    read.table(
        "outside_nohit.bed", 
        sep="\t",
        col.names=c("chr","start","stop","id","score","strand")) %>%
        mutate(source="outside",hit=F)
) %>%
select(id,score,source,hit) %>%
rowwise() %>%
#will be evaluated in external plotting function
#mutate(score=max(0,score)) %>%
mutate(hasdistance=T)

d %>% summary

In [ ]:
%%R

# fractions of Alus with and without distance

d %>%
group_by(source, hit) %>%
summarise(
    fraction_paired = sum(hasdistance) / n(),
    n_paired = sum(hasdistance),
    n = n())

In [ ]:
%%R -h 400 -w 800 -u px

# compare distances of alus with and without DHX9 binding site

typestr <- data_frame(
    hit=c(T,
          F,
          T,
          F), 
    source=c("genes",
             "genes",
             "outside",
             "outside"), 
    type=c(
        "DHX9-targeted Alu on gene", 
        "non-targeted Alu on gene",
        "DHX9-targeted Alu outside of gene",
        "non-targeted Alu outside of gene"))

typestr %>% print

In [ ]:
%%R

d_plotaludist <- inner_join(
    filter(d,hasdistance),
    typestr,
    by=c("source", "hit"))

In [ ]:
%%R -h 400 -w 800 -u px

library(ggplot2)

p_aludist <- ggplot(d_plotaludist) + 
geom_density(aes(score, color=type, linetype=hit)) +
ggtitle("Strong DHX9 signal is associated with close complementary Alu pairs") +
xlab("distance to next reverse-complementary Alu")

ggsave("paired_alu_distances.png", h=4, w=7)
ggsave("paired_alu_distances.pdf", h=4, w=7)

p_aludist + scale_x_log10(limits=c(1,10000))

In [ ]:
%%R

# summary

d_plotaludist %>%
group_by(hit, type) %>%
summarise(
    avg=mean(score), 
    median=median(score),
    count=n()) %>% print

In [ ]:
%%R

d_plotaludist %>% head %>% print

filter(d_plotaludist, hit, source=="gene") %>% nrow

In [ ]:
%%R

# compare hit distances ongene and offgene

t <- filter(d_plotaludist, hit)

wilcox.test(t$score~t$source) 

In [ ]:
%%R

# compare hit distances hit and nonhit ongene

t <- filter(d_plotaludist, source=="genes")

wilcox.test(t$score~t$hit) 

In [ ]:
%%R

# write table for further plotting

write.table(
    mutate(d_plotaludist, experiment="uvCLAP DHX9"),
    "aludists_uvCLAP-DHX9.csv",
    quote=F,
    sep="\t",
    row.names=F)